# **Objective**
This is to show how to submit the trained model with a sample trained model (>83%).
Refer to the following notebooks for better models. I learned a lot from them.

> **Reference**
> >   Cassava Disease 🌿 Keras + TF + EfficentNet (90%)<br>
> >     https://www.kaggle.com/andreshg/cassava-disease-keras-tf-efficentnet-90
> 
> >   Cassava EfficientNetB3 Beginner (88%) <br>
> >     https://www.kaggle.com/khanrahim/cassava-efficientnetb3-beginner-88

# **Prerequisite**
Download "model_sample.h5" from https://www.kaggle.com/matsumuranaoki/samplemodel <br>
and upload by clicking "Add data" in the "Data" section on the top right of your notebook. 

# **Instruction**
1. In the code, modify the following file path accordingly.<br>
   final_model = models.load_model('../input/samplemodel/model_sample.h5')
2. Turn off "Internet" in the "Setting" section on the right of your notebook.
3. Save & Run All (Commit) the code on your notebook. "submission.csv" will be created in "output" folder in "Data" section.
4. On the "Cassava Leaf Disease Classification" page, click "Submit Predictions" and select your notebook, version and output file (submission.csv).

Please note that submission would take ~2 hours as it uses different test set which contains many data.



In [ ]:
from keras import models
from keras_preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

train_csv_location = '../input/cassava-leaf-disease-classification/train.csv'
train_image_location = '../input/cassava-leaf-disease-classification/train_images'
test_image_location = '../input/cassava-leaf-disease-classification/test_images'

traindf = pd.read_csv(train_csv_location, dtype=str)
test_image_dir = os.listdir(test_image_location)
testdf = pd.DataFrame(test_image_dir, columns=['image_id'])

train, valid = train_test_split(traindf, test_size = 0.05, 
    random_state = 42, stratify = traindf['label']
    )

IMG_SIZE = 456
size = (IMG_SIZE,IMG_SIZE)
n_CLASS = 5
BATCH_SIZE = 15

datagen_train = ImageDataGenerator(
    preprocessing_function= tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=40,
    width_shift_range= 0.2,
    height_shift_range= 0.2,
    shear_range= 0.2,
    zoom_range= 0.2,
    horizontal_flip= True,
    vertical_flip= True,
    fill_mode='nearest'
)

test_datagen=ImageDataGenerator(
    preprocessing_function= tf.keras.applications.efficientnet.preprocess_input,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe = train,
    directory = train_image_location,
    seed= 42,
    x_col = 'image_id',
    y_col = 'label',
    target_size= size,
    class_mode= 'categorical',
    interpolation= 'nearest',
    shuffle= True,
    batch_size= BATCH_SIZE
)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory= test_image_location,
    seed=42,
    x_col = 'image_id',
    y_col = None,
    target_size= size,
    interpolation= 'nearest',
    shuffle= False,
    batch_size=1,
    class_mode=None,
)
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

# evaluate the best model
final_model = models.load_model('../input/samplemodel/model_sample.h5')

# predict output
test_generator.reset()
pred = final_model.predict(test_generator, steps = STEP_SIZE_TEST, verbose = 1)

predicted_class_indices=np.argmax(pred,axis=1)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

# export to csv
filenames=test_generator.filenames
results=pd.DataFrame({"image_id":filenames, "label":predictions})
results.to_csv("submission.csv",index=False)